In [35]:
import torch
from torch.utils.data import TensorDataset, DataLoader, Subset

# data = torch.load('data_to_train/tensors.pt')
data = torch.load('temp/tensors.pt')
data.shape

/tmp/ipykernel_79154/4075737016.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('temp/tensors.pt')


torch.Size([262, 26, 25, 7])

In [44]:
def create_dataset(data):
    labels = data[..., 0]
    features = data[..., 1:]
    features = features.permute(0, 3, 1, 2).contiguous()
    dataset = TensorDataset(features, labels)
    return dataset


def train_val_test_split(dataset, val_indices, test_indices):
    all_indices = set(range(len(dataset)))
    train_indices = list(all_indices - set(val_indices) - set(test_indices))
    train_indices.sort()
    train_dataset = Subset(dataset, train_indices)
    val_dataset = Subset(dataset, val_indices)
    test_dataset = Subset(dataset, test_indices)
    return train_dataset, val_dataset, test_dataset

In [45]:
dataset = create_dataset(data)
val_indices = [20, 33, 52, 71, 90, 110, 132, 155, 177, 197, 219, 237, 250, 257, 22, 39, 58, 77, 96, 116, 138, 161, 183, 203, 225, 243]
test_indices = [21, 34, 53, 72, 91, 111, 133, 154, 178, 198, 220, 238, 251, 258, 23, 40, 59, 78, 97, 117, 139, 162, 184, 204, 226, 244]
train_dataset, val_dataset, test_dataset = train_val_test_split(dataset, val_indices, test_indices)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Train dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))
print("Test dataset size:", len(test_dataset))

Train dataset size: 210
Validation dataset size: 26
Test dataset size: 26


In [54]:
i, j = train_dataset[0]
print(i.shape)
print(j.shape)

torch.Size([6, 26, 25])
torch.Size([26, 25])


In [56]:
print(j)

tensor([[55., 50., 50., 50., 50., 45., 45., 45., 45., 45., 45., 45., 45., 65.,
         60., 55., 25.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [55., 50., 50., 50., 50., 50., 45., 45., 45., 45., 45., 45., 50., 65.,
         60., 55., 10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [50., 50., 50., 50., 50., 50., 45., 45., 45., 45., 40., 45., 60., 65.,
         55., 50.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [50., 50., 50., 50., 50., 50., 45., 45., 45., 45., 40., 50., 60., 60.,
         55., 30.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [50., 50., 50., 50., 50., 45., 45., 45., 45., 45., 45., 55., 65., 60.,
         55., 25.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [50., 50., 50., 45., 45., 45., 45., 45., 45., 45., 55., 55., 65., 60.,
         55., 15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [50., 50., 50., 45., 45., 45., 45., 45., 45., 50., 55., 65., 65., 55.,
         55., 15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim

# # Example input data and labels
# batch_size, height, width, channels = 8, 32, 32, 3
# num_classes = 10

# input_data = torch.randn(batch_size, channels, height, width)
# labels = torch.randint(0, 2, (batch_size, height, width))

# # Create a binary mask
# mask = (labels != 0).float()
# mask = mask.unsqueeze(1)  # Expand dimensions to match input data

# # Apply the mask to the input data
# masked_input = input_data * mask

# # Define the CNN model
# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv2d(channels, 32, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#         self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
#         self.fc1 = nn.Linear(64 * (height // 4) * (width // 4), 64)
#         self.fc2 = nn.Linear(64, num_classes)

#     def forward(self, x):
#         x = self.pool(torch.relu(self.conv1(x)))
#         x = self.pool(torch.relu(self.conv2(x)))
#         x = torch.relu(self.conv3(x))
#         x = x.view(-1, 64 * (height // 4) * (width // 4))
#         x = torch.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

# # Instantiate the model, define the loss function and the optimizer
# model = CNN()
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Training loop
# num_epochs = 10
# for epoch in range(num_epochs):
#     optimizer.zero_grad()
#     outputs = model(masked_input)
#     loss = criterion(outputs, labels.view(-1))
#     loss.backward()
#     optimizer.step()
#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SegmentationCNN(nn.Module):
    def __init__(self, in_channels=6, num_classes=18):
        super(SegmentationCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, num_classes, kernel_size=3, padding=1)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)
        return x

# Przykładowe dane wejściowe
input_tensor = torch.randn(1, 6, 26, 25)
model = SegmentationCNN()
output = model(input_tensor)
print(output.shape)  # Powinno być torch.Size([1, 18, 26, 25])

# Przykładowe etykiety
labels = torch.tensor([[55., 50., 50., 50., 50., 45., 45., 45., 45., 45., 45., 45., 45., 65.,
                        60., 55., 25.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
                       [55., 50., 50., 50., 50., 50., 45., 45., 45., 45., 45., 45., 50., 65.,
                        60., 55., 10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
                       [50., 50., 50., 50., 50., 50., 45., 45., 45., 45., 40., 45., 60., 65.,
                        55., 50.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
                       [50., 50., 50., 50., 50., 50., 45., 45., 45., 45., 40., 50., 60., 60.,
                        55., 30.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
                       [50., 50., 50., 50., 50., 45., 45., 45., 45., 45., 45., 55., 65., 60.,
                        55., 25.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
                       [50., 50., 50., 45., 45., 45., 45., 45., 45., 45., 55., 55., 65., 60.,
                        55., 15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
                       [50., 50., 50., 45., 45., 45., 45., 45., 45., 50., 55., 65., 65., 55.,
                        55., 15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
                       [50., 50., 45., 45., 45., 45., 45., 45., 50., 50., 55., 65., 60., 55.,
                        55., 15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
                       [50., 45., 45., 45., 45., 45., 40., 45., 45., 50., 55., 65., 60., 55.,
                        55., 10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
                       [45., 45., 45., 45., 45., 40., 40., 45., 50., 55., 55., 65., 60., 55.,
                        55.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
                       [15., 20., 25., 25., 10.,  5., 10., 45., 50., 55., 55., 65., 60., 55.,
                        55.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
                       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 40., 50., 55., 55., 65., 60., 55.,
                        55.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
                       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 30., 50., 50., 55., 65., 60., 55.,
                        55., 15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
                       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 35.,
                        25.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=torch.float64)

# Konwersja etykiet do typu long
labels = labels.long()

# Maskowanie klasy zerowej
mask = labels != 0

# Przykładowa funkcja straty
criterion = nn.CrossEntropyLoss()

# Przykładowe wyjście modelu
output = model(input_tensor)

# Przekształcenie wyjścia modelu i etykiet do odpowiednich kształtów
output = output.permute(0, 2, 3, 1).contiguous().view(-1, 18)
labels = labels.view(-1)

# Zastosowanie maski
# output = output[mask.view(-1)]
# labels = labels[mask.view(-1)] - 1  # Przesunięcie etykiet o 1, aby zaczynały się od 0

# Obliczenie straty
loss = criterion(output, labels)
print(loss.item())

torch.Size([1, 18, 26, 25])


ValueError: Expected input batch_size (650) to match target batch_size (350).